## Programmation concurente - <span style="color:blue;">CORRECTION</span>

**Sources** : Numérique et Sciences Informatiques - <span class="codepy">ellipses</span>

**Ressources** : <a href="https://webge.synology.me/dokuwiki/doku.php?id=python:accueilpython" target="_blank"><input type="button" value="Wiki Python sur WebGE"></a> 

### Sommaire
+ **Introduction** <br>
+ **Exemple 1** - Comptage en parallèle  <br>
+ **Exemple 2** - Compteur global partagé (problème de concurrence)  <br>
+ **Exemple 3** - Interblocage

### Introduction
#### Les processus
<em>Grace à leur système d'exploitation **multitâche**, les ordinateurs exécutent de façon **concurrente** plusieurs programmes.  L'exécution d'un programme s'appelle un **processus**. C'est le système d'exploitation, et en particulier l'**ordonnanceur** (une des fonctionnalités du **noyau**), qui détermine quel processus s'exécute à un instant donné. Le fait pour un processus d'être interrompu s'appelle une **commutation de contexte**. Plusieurs processus s'exécutant de façon concurrente peuvent s'**interbloquer** s'ils attendent de pouvoir accéder à un même ensemble de **ressources en accès exclusif**. </em><br>
<img src="img/se.png"> <br>
#### Les threads
<em>Les **threads** ou processus légers sont des "sous-processus", démarrés par un processus et s'exécutant de manière concurrente avec le reste du programme. L'accès à des ressources par plusieurs threads peut être protégé par des **verrous**. Une portion de code comprise entre l'acquisition et le relâchement d'un verrou s'appelle une **section critique**. <br>
Le module threading de la bibliothèque standard Python permet de démarrer des threads.</em><br>
<img src="img/threads.png"> 

### Objectifs
S'initier à la **programmation multithread** en Python et **illustrer les problèmes de concurence et d'interblocage** dans un cadre plus contrôlé que dans un système d'exploitation.

### Exemple 1 - Comptage en parallèle
<em>Dans le code ci-dessous, le programme principal effectue une boucle et appelle quatre fois l'expression <span style="font-family:Consolas;font-weight:bold;font-style:normal">threading.Thread(target=hello, arg=[n])</span>. Cette dernière crée un objet de type **Thread**. La variable <span class="code">t</span> contient l'objet **Thread** créé. La méthode .start() lance l'exécution de la fonction en tâche de fond. Cette méthode rend directement la main et le programme principal continue de s'exécuter de façon concurente au thread démarré. Une fois la boucle exécutée, ce programme comporte cinq *threads* : ceux démarrés par .start() et le programme principal.</em>

In [ ]:
# Programmation concurente - Comptage 
import threading

def hello(num):
    for i in range(5):
        print(f"je suis le thread {num} et ma valeur est {i}")
    print(f"----- Fin du thread {num} -----")


for n in range(4):
    t = threading.Thread(target=hello, args=[n]) # l'argument de type target est une fonction et l'argument arg un tableau
    t.start()                                    # d'arguments passés à la fonction

> **Activité 1. Exécutez** plusieurs fois le code ci-dessus. Que remarquez-vous ?

<p style="color:blue; font-weight:bold">Correction</p>
<ul style="color:blue;">
    <li>Les threads alternent leur exécution au gré des commutation de contexte.</li>
    <li>Deux exécutions successives donnent des affichages différents.</li>
</ul>

**Remarque** : L'ordre dans lequel sont démarrés les threads ne donne aucune indication sur l'ordre dans lequel ils peuvent se terminer.

> **Activité 2. En vous inspirant du programme ci-dessus, écrivez** un programme qui exécute **quatre** fonctions concurrentes affichant plusieurs fois un message de bienvenu personnalisé. Une fonction produira un maximum de **dix** fois le message.<br>

> *Exemple de résultats attendus*<br>
Bonjour, je suis le thread 0 et ceci est le message 1<br>
... <br>
Message de bienvenu du thread 1 qui transmet son message 3<br>
... <br>


In [ ]:
# Programmation concurente - Messages différents
# Correction

import threading
'''
num : numéro du thread
msg : message affiché
nb : nombre de message
'''
def hello(num,msg,nb):
    for i in range(nb):
        print(f"{msg} {i}")
    print(f"----- Fin du thread {num} -----")
    
nb=[10,7,5,8]
msg=["Bonjour, je suis le thread 0 et ceci est le message ","Message de bienvenu du thread 1 qui transmet le message ",
     "Salut, le thread 2 vous envoie le message ", "Hé, le thread 3 aussi envoie son message "]

for num in range(len(nb)):
    t = threading.Thread(target=hello, args=[num,msg[num], nb[num]]) 
    t.start()

### Exemple 2 - Compteur global partagé (problème de concurrence)
Les threads peuvent servir à illustrer les problèmes de concurrence. Le programme ci-dessous définit une variable globale COMPTEUR. Comme 'hello' dans le programme précédent, la fonction incrc s'exécute dans des Threads. Le programme principal déclare un tableau vide th. Il démarre ensuite quatre threads et stocke les objets correspondants dans le tableau th, après les avoir démarrés. Pour chacun des threads stockés, la méthode join est appelée. Cette méthode permet d'attendre que le thread auquel on l'applique soit terminé. Si le thread est déja terminé, la méthode se termine immédiatement.


> **Activité 3. Analyse** du programme <br>
> Que fait la fonction incrc ? Quel devrait être la valeur de COMPTEUR à la fin du programme ? 

<p style="color:blue; font-weight:bold">Correction</p>
<ul style="color:blue;">
    <li>La fonction incrc exécute 100000 itérations d'une boucle qui incrémente la variable COMPTEUR.</li>
    <li>A la fin du programme, la variable COMPTEUR devrait être égale à 400000.</li>
</ul>

> **Activité 4. Exécutez** plusieurs fois le code ci-dessous. Que remarquez-vous ?

In [ ]:
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence
import threading

COMPTEUR = 0

def incrc():
    global COMPTEUR
    for _ in range(100000):
        v = COMPTEUR      # ces deux ligne pourraient être remplacées par COMPTEUR +=1. 
        COMPTEUR = v + 1  # Elle sont utilisée ici pour simplifiées les explications

th=[]
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()
print(f"Valeur finale = {COMPTEUR}") # Cette ligne est exécutée lorsque tous les threads sont terminés

**REMARQUE** : Contrairement à ce que l'on peut observer sur quelques essais, le programme ci-dessus ne fonctionne pas dans la durée.

> **Activité 5. Modifiez** le programme ci-dessous pour qu'il affiche les résultats faux. <br>

> *Exemple de résultat attendu*<br>
En attente d'une erreur<br>
Valeur finale de COMPTEUR erronée = 300000 après 2643 tests<br>
Valeur finale de COMPTEUR erronée = 300000 après 10504 tests<br>
Valeur finale de COMPTEUR erronée = 393134 après 13748 tests<br>

In [ ]:
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence
# Correction
import threading
COMPTEUR=0
i=1

def incrc():
    global COMPTEUR
    for c in range(100000):
        v = COMPTEUR
        COMPTEUR = v + 1

print("En attente d'une erreur !")

while True:
    th=[]
    for n in range(4):
        t = threading.Thread(target=incrc, args=[])
        t.start()
        th.append(t)
    for t in th:
        t.join()
    i+=1
    if COMPTEUR != 400000:
    print(f"Valeur finale de COMPTEUR = {COMPTEUR} après {i} tests") 
    COMPTEUR=0

### Correction du problème de concurrence dans le programme "Compteur partagé"
Pour corriger ce problème, il faut garantir l'**accès EXCLUSIF** à la variable compteur entre sa lecture et son écriture. On peut pour cela utiliser un verrou. Un **verrou** est un objet que l'on essaye d'acquérir. Si on est le premier à faire cette demande, on acquiert le verrou. On peut le rendre à tout moment. Si en revanche quelqu'un d'autre tient le verrou alors on est bloqué jusqu'à ce qu'il soit libéré. Des verrous munis de ces deux opérations sont disponibles dans le **module threading** avec le constructeur **Lock()**. Une fois le verrou construit, on peut tenter de l'acquérir avec la méthode **acquire()** et on peut le rendre avec la méthode **release()**. 

**Note** : Une portion de code protégée par un verrou s'appelle une **SECTION CRITIQUE**.

> **Activité 5. Exécutez** le programme ci-dessous. Laissez le programme atteindre 5000 simulation avant de répondre au questions ci-dessous. En attendant passez à la suite.

> _Après 5000 simulations_ : <br>
a) Que remarquez-vous ?<br>
b) Expliquez pourquoi on a corriger le problème de concurence entre les treads t1, t2, t3 et t4.

<p style="color:blue; font-weight:bold">Correction</p>
<ul style="color:blue;">
    <li>a) COMPTEUR = 40000, le résultat est celui attendu.</li>
    <li>b) Un thread ne peut pas incrémenté le compteur s'il ne dispose pas du verrou. Un seul thread peut disposer du verrou. Pour qu'un thread se termine, il doit avoir incrémenté le compteur 10000 fois. </li>
</ul>

In [ ]:
# Programmation concurente - Compteur partagé
# Correction du problème de concurrence avec un verrou

import threading

COMPTEUR = 0
verrou = threading.Lock() # construction du verrou
i=1

def incrc():
    global COMPTEUR
    for c in range(100000):
        verrou.acquire() # Acquisition du verrou
        v = COMPTEUR
        COMPTEUR = v + 1
        verrou.release() # Relâchement du verrou
 
while True:
    th=[]
    for n in range(4):
        t = threading.Thread(target=incrc, args=[])
        t.start()
        th.append(t)
    for t in th:
        t.join()
    i+=1
    print(f"Simulation{i}, COMPTEUR = {COMPTEUR}") 
    COMPTEUR=0

### Exemple 3 - Interblocage
L'utilisation de plusieurs verrous rend les **interblocages** possibles.

> **Activité 6. Analyse** du programme ci-dessous <br>
Quel pourrait être le texte affiché par le programme : <br>
a) S'il ne se bloquait pas ? <br>
b) S'il se bloquait ? 

<p style="color:blue; font-weight:bold">Correction</p>
<ul style="color:blue;">a) Pas d'interblocage
    <li>f1 a acquit le verrou 1</li>   
    <li>f1 a acquit le verrou 2</li> 
    <li>f1 a relâché le verrou 2</li> 
    <li>f1 a relâché le verrou 1</li> 
    <li>f2 a acquit le verrou 2</li> 
    <li>f2 a acquit le verrou 1</li>
    <li>f2 a relâché le verrou 1</li>
    <li>f2 a relâché le verrou 2</li>
</ul>
<ul style="color:blue;">a) Interblocage
    <li>f1 a acquit le verrou 1</li>   
    <li>f1 a acquit le verrou 2</li> 
</ul>

In [ ]:
# Interblocage
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def f1():
    verrou1.acquire()
    print("f1 a acquit le verrou 1")
    i=0; j=0
    for _ in range(10000):
        i+=1
        for _ in range (1000):
          j+=1  
    verrou2.acquire()
    print("f1 a acquit le verrou 2")
    verrou2.release()
    print("f1 a relâché le verrou 2")
    verrou1.release()
    print("f1 a relâché le verrou 1")
    
    
def f2():
    verrou2.acquire()
    print("f2 a acquit le verrou 2")
    verrou1.acquire()
    print("f2 a acquit le verrou 1")
    verrou1.release()
    print("f2 a relâché le verrou 1")
    verrou2.release()
    print("f2 a relâché le verrou 2")

t1 = threading.Thread(target=f1, args=[])
t2 = threading.Thread(target=f2, args=[])
t1.start()
t2.start()

> **Activité 7. Diminuez** la durée des boucles pour que le programme ne se bloque plus. Peut-on conserver cette solution ?

<p style="color:blue; font-weight:bold">Correction</p>
<ul style="color:blue;">
    <li></li> 
</ul>

**A retenir**